In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.time_series import *
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [10]:
# Load the data
spurs = pd.read_csv('..\\data\\spurs.csv')
spurs = spurs[(spurs['season_type']==0) & (spurs['season']<2021) & (spurs['season']>2000)].reset_index(drop=True)

## Econometrics

In [11]:
spurs=spurs[['score']]
exp = TSForecastingExperiment()
exp.setup(data = spurs, target='score' ,  fh = 82, coverage=0.90)

,Description,Value
0,session_id,7746
1,Target,score
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(1531, 1)"
5,Transformed data shape,"(1531, 1)"
6,Transformed train set shape,"(1449, 1)"
7,Transformed test set shape,"(82, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [12]:
exp.check_stats(test='adf')

,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,ADF,Transformed,Stationarity,{'alpha': 0.05},True
1,Stationarity,ADF,Transformed,p-value,{'alpha': 0.05},0.001506
2,Stationarity,ADF,Transformed,Test Statistic,{'alpha': 0.05},-3.982301
3,Stationarity,ADF,Transformed,Critical Value 1%,{'alpha': 0.05},-3.434671
4,Stationarity,ADF,Transformed,Critical Value 5%,{'alpha': 0.05},-2.863448
5,Stationarity,ADF,Transformed,Critical Value 10%,{'alpha': 0.05},-2.567786


In [13]:
exp.plot_model(plot="diff", data_kwargs={"order_list": [1], "acf": True, "pacf": True})

In [15]:
auto_arima = exp.create_model('auto_arima', cross_validation=False)
auto_ets = exp.create_model('ets', cross_validation=False)
arima111 = exp.create_model('arima', order = (1,1,1), seasonal_order=(0,0,0,4), with_intercept=True, cross_validation=False) 

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.7325,0.7192,8.8514,10.9922,0.0768,0.0780,-0.0555


,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.7381,0.7108,8.9200,10.8643,0.0781,0.0786,-0.0311


,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.7228,0.7030,8.7347,10.7442,0.0766,0.0769,-0.0085


In [16]:
exp.compare_models(include=[arima111, auto_arima, auto_ets], cross_validation=False)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
0,ARIMA,0.7228,0.7030,8.7347,10.7442,0.0766,0.0769,-0.0085,0.4500
1,Auto ARIMA,0.7325,0.7192,8.8514,10.9922,0.0768,0.0780,-0.0555,17.0000
2,ETS,0.7381,0.7108,8.9200,10.8643,0.0781,0.0786,-0.0311,0.1600


ARIMA(order=(1, 1, 1), seasonal_order=(0, 0, 0, 4))

In [17]:
exp.plot_model([ auto_ets], plot='forecast', data_kwargs={'fh':82, 'labels':["Auto ETS"]})

In [18]:
exp.plot_model([ auto_arima], plot='forecast', data_kwargs={'fh':82, 'labels':["Auto ETS"]})

## Machine Learning

In [9]:
spurs['Lag1'] = spurs['score'].shift(1)
spurs['Lag2'] = spurs['score'].shift(2)
spurs['Lag3'] = spurs['score'].shift(3)

X = spurs[['Lag1', 'Lag2', 'Lag3']]
y = spurs['score']

# Split the data
# use last season as test
test_range=82


X_train, X_test = X[:-test_range], X[-test_range:]
y_train, y_test = y[:-test_range], y[-test_range:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1449, 3), (82, 3), (1449,), (82,))

In [ ]:
def forecast_future(model, data, steps=test_range):

    future_forecasts = np.empty(steps)

    lag1=data['Lag 1'].iloc[-1]
    lag2=data['Lag 2'].iloc[-1]
    lag3=data['Lag 3'].iloc[-1]

    for i in range(steps):
        forecast = model.predict([[lag1, lag2, lag3]])[0]
        future_forecasts[i]=(forecast)

        lag3=lag2
        lag2=lag1
        lag1=forecast
    
    return future_forecasts



In [21]:
exp = TSForecastingExperiment()
exp = setup(data = spurs, target = 'score', fh = 24, session_id=1000)
exp.compare_models(cross_validation=False, sort='rmse')

,Description,Value
0,session_id,1000
1,Target,score
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(1531, 1)"
5,Transformed data shape,"(1531, 1)"
6,Transformed train set shape,"(1507, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
croston,Croston,0.6153,0.5832,7.3867,8.8530,0.0643,0.0644,-0.0086,0.0200
ets,ETS,0.6309,0.5910,7.5735,8.9715,0.0658,0.0660,-0.0358,0.1400
theta,Theta Forecaster,0.6360,0.5948,7.6348,9.0294,0.0662,0.0666,-0.0492,0.0400
auto_arima,Auto ARIMA,0.6353,0.5984,7.6268,9.0849,0.0658,0.0665,-0.0621,13.8800
exp_smooth,Exponential Smoothing,0.6467,0.6012,7.7630,9.1276,0.0677,0.0677,-0.0722,0.2100
polytrend,Polynomial Trend Forecaster,0.7482,0.7030,8.9816,10.6722,0.0755,0.0787,-0.4657,0.0200
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.7536,0.7081,9.0471,10.7495,0.0763,0.0793,-0.4870,0.0900
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.7536,0.7081,9.0471,10.7495,0.0763,0.0793,-0.4870,0.0600
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.7542,0.7086,9.0537,10.7569,0.0763,0.0794,-0.4891,0.0700
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.7546,0.7089,9.0581,10.7624,0.0764,0.0794,-0.4906,0.0700


Croston()